# Lab 3 PySpark

## Задание 1: Теория

### Что такое PySpark и в чем его главное отличие от Pandas?

🔥 PySpark

**PySpark** — это Python API для **Apache Spark**, фреймворка распределённых вычислений.  
Он позволяет обрабатывать **огромные объёмы данных (терабайты и больше)**, распределяя задачи между множеством машин (кластером).

Используется для:
- ETL-процессов (извлечение, трансформация, загрузка);
- анализа больших данных;
- машинного обучения (через `pyspark.ml`);
- выполнения SQL-запросов на кластере.

---

🐼 Pandas

**Pandas** — библиотека Python для **анализa табличных данных в оперативной памяти**.  
Она удобна для:
- обработки **небольших и средних** наборов данных (до нескольких ГБ);
- **интерактивного анализа** и **прототипирования**;
- интеграции с NumPy, Matplotlib, scikit-learn и другими библиотеками.

---

⚖️ Главное отличие: масштаб и архитектура

| Критерий | **Pandas** | **PySpark** |
|-----------|-------------|--------------|
| **Тип вычислений** | Однопоточные, локальные | Распределённые, параллельные |
| **Где работает** | На одном компьютере | На кластере Spark |
| **Размер данных** | Влезает в оперативную память | Может превышать объём памяти |
| **API** | DataFrame (локальный) | DataFrame (распределённый) |
| **Производительность** | Быстрее на малых данных | Эффективнее на больших |
| **Ошибки памяти (OOM)** | Возможны | Отсутствуют (данные делятся на партиции) |
| **Установка** | `pip install pandas` | Требуется Spark (`pyspark` + кластер или локальный Spark) |

### Что такое PySpark MLlib и в чем ее главное отличие от Sklearn?

🔥 PySpark MLlib

**PySpark MLlib** — это библиотека **машинного обучения** для **Apache Spark**, предназначенная для **распределённой обработки данных**.  
Она позволяет обучать и применять модели машинного обучения **на кластере**, обрабатывая **большие объёмы данных**, которые не помещаются в оперативную память одного компьютера.

Возможности MLlib:
- Классические алгоритмы машинного обучения (регрессия, классификация, кластеризация);
- Обработка признаков: нормализация, one-hot encoding, TF-IDF и др.;
- Pipeline API для сборки конвейеров обучения;
- Интеграция с Spark SQL и DataFrame API;
- Распределённое обучение на многих узлах.

🧠 Scikit-learn (sklearn)

**Scikit-learn** — это библиотека машинного обучения для **локальной обработки данных** (на одном компьютере).
Она широко используется для **прототипирования, обучения моделей и анализа данных**, когда объём данных помещается в оперативную память.

Возможности sklearn:

* Большое количество моделей (от линейных до ансамблевых);
* Простое и интуитивное API (`fit`, `predict`, `score`);
* Интеграция с Pandas и NumPy;
* Поддержка пайплайнов и кросс-валидации.

---

⚖️ Главное отличие: масштаб и архитектура

| Критерий              | **Scikit-learn**                    | **PySpark MLlib**                |
| --------------------- | ----------------------------------- | -------------------------------- |
| **Тип вычислений**    | Локальные, однопроцессорные         | Распределённые, кластерные       |
| **Размер данных**     | До нескольких ГБ (влезает в память) | До терабайт и выше               |
| **Где работает**      | На одном компьютере                 | На Spark-кластере                |
| **Скорость**          | Быстрее на малых данных             | Эффективнее на больших           |
| **Поддержка GPU**     | Частично (через другие пакеты)      | Нет (в основном CPU-кластеры)    |
| **API**               | Python (NumPy/Pandas)               | PySpark DataFrame                |
| **Pipeline API**      | Есть (`Pipeline`)                   | Есть (`pyspark.ml.Pipeline`)     |
| **Тип использования** | Исследования, прототипы             | Big Data и продакшн ML-пайплайны |

### В чем концепция "Ленивых вычислений" (Lazy Evaluation) в PySpark?

Что такое ленивые вычисления

**Ленивые вычисления (Lazy Evaluation)** — это принцип, по которому **PySpark не выполняет операции сразу**, как только вы их вызываете.  
Вместо этого Spark **строит план вычислений** (называемый *DAG — Directed Acyclic Graph*),  
а **реальное выполнение** происходит **только тогда**, когда вызывается *действие* (*action*).

---
🧠 Суть концепции

```python
df = spark.read.csv("data.csv", header=True, inferSchema=True)
filtered = df.filter(df["age"] > 30)
grouped = filtered.groupBy("country").count()
````

👉 На этом этапе **ничего не вычисляется**.
Spark **только запоминает**, какие шаги нужно выполнить:

1. Прочитать файл `data.csv`;
2. Отфильтровать строки, где `age > 30`;
3. Сгруппировать по `country` и посчитать количество.

Реальные вычисления начнутся **только при вызове действия**, например:

```python
grouped.show()
```

---

🪄 Почему это полезно

| Преимущество                            | Описание                                                                                                                     |
| --------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------- |
| **Оптимизация**                         | Spark может проанализировать весь план вычислений и **оптимизировать** его (например, объединить несколько фильтров в один). |
| **Минимизация чтений и передач данных** | Spark выполняет операции только тогда, когда это нужно, избегая лишних обращений к диску и сети.                             |
| **Повышение производительности**        | Позволяет эффективно использовать ресурсы кластера, минимизируя количество этапов (stages).                                  |

---

🧩 Действия (Actions) и Трансформации (Transformations)

| Тип операции                        | Примеры                                             | Что делает                                         |
| ----------------------------------- | --------------------------------------------------- | -------------------------------------------------- |
| **Трансформации (Transformations)** | `select`, `filter`, `groupBy`, `join`, `withColumn` | Создают новый DataFrame, **не выполняются сразу**  |
| **Действия (Actions)**              | `show`, `count`, `collect`, `save`, `write`         | **Запускают выполнение** всех накопленных операций |

### В чем разница между "Трансформациями" (Transformations) и "Действиями" (Actions)?

🧩 Общая идея

В PySpark все операции над данными делятся на два типа:

1. **Трансформации (Transformations)** — создают *новое представление данных*,  
   но **не выполняются немедленно** (следуют принципу *ленивых вычислений*).

2. **Действия (Actions)** — **запускают выполнение** всех накопленных трансформаций  
   и **возвращают результат** пользователю или записывают его на диск.

---

🧱 1. Трансформации (Transformations)

**Трансформации** — это операции, которые **создают новый DataFrame** на основе существующего.  
Они *описывают*, какие преобразования нужно сделать, но **не выполняются сразу**.

📘 Примеры трансформаций:
```python
df2 = df.filter(df["age"] > 30)
df3 = df2.select("name", "age")
df4 = df3.groupBy("age").count()
````

🔹 Каждая строка **создаёт новый DataFrame**,
но Spark только строит **план вычислений (DAG)** — ничего реально не делает до вызова действия.

💡 Примеры трансформаций:

| Тип операции       | Примеры                               |
| ------------------ | ------------------------------------- |
| Фильтрация         | `filter()`, `where()`                 |
| Выбор колонок      | `select()`, `drop()`                  |
| Агрегация          | `groupBy()`, `agg()`                  |
| Объединение        | `join()`, `union()`                   |
| Добавление колонок | `withColumn()`, `withColumnRenamed()` |
| Сортировка         | `sort()`, `orderBy()`                 |

---

🚀 2. Действия (Actions)

**Действия** — это операции, которые **триггерят выполнение** всех накопленных трансформаций
и возвращают результат (или записывают его).

📘 Примеры действий:

```python
df4.show()        # показать несколько строк
df4.count()       # посчитать количество строк
df4.collect()     # вернуть все данные в виде списка Row
df4.write.csv("out.csv")  # записать результат на диск
```

🔹 Когда вы вызываете действие, Spark:

1. анализирует весь план трансформаций;
2. оптимизирует его (через Catalyst Optimizer);
3. выполняет вычисления на кластере.

💡 Примеры действий:

| Тип операции              | Примеры                          |
| ------------------------- | -------------------------------- |
| Просмотр результата       | `show()`, `head()`               |
| Агрегация                 | `count()`, `first()`, `take()`   |
| Получение данных в Python | `collect()`, `toPandas()`        |
| Сохранение результата     | `write.csv()`, `write.parquet()` |

---

⚖️ Сравнительная таблица

| Критерий                  | **Трансформации (Transformations)** | **Действия (Actions)**                 |
| ------------------------- | ----------------------------------- | -------------------------------------- |
| **Когда выполняются**     | Лениво (отложенно)                  | Немедленно                             |
| **Что возвращают**        | Новый DataFrame                     | Результат (число, список, файл и т.д.) |
| **Изменяют ли данные**    | Нет (создают новое представление)   | Да (выполняют вычисления)              |
| **Пример**                | `filter()`, `select()`, `groupBy()` | `show()`, `count()`, `collect()`       |
| **Запускают ли план DAG** | Нет                                 | Да                                     |

### Что такое RDD, DataFrame и SparkSession?

🧩 1. RDD (Resilient Distributed Dataset)

💡 Определение
**RDD** — это **основная низкоуровневая структура данных** в Spark,  
представляющая **распределённый набор элементов**, разделённых по узлам кластера.

RDD — это **иммутабельная** (неизменяемая) коллекция объектов,  
над которой можно выполнять **распределённые параллельные операции**.

---

⚙️ Основные характеристики RDD:
| Свойство | Описание |
|-----------|-----------|
| **Resilient** | Восстанавливается после сбоев (Spark хранит lineage — историю преобразований) |
| **Distributed** | Распределён между узлами кластера |
| **Dataset** | Набор данных (коллекция элементов) |
| **Immutability** | После создания RDD нельзя изменить, только создать новое на его основе |
| **Lazy Evaluation** | Трансформации выполняются лениво, при вызове действия |

---

🔧 Пример работы с RDD
```python
from pyspark import SparkContext

sc = SparkContext("local", "RDD Example")

rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd2 = rdd.map(lambda x: x * 2)
rdd3 = rdd2.filter(lambda x: x > 5)

print(rdd3.collect())
# [6, 8, 10]
````

📘 Здесь данные `[1, 2, 3, 4, 5]` распределены по узлам,
а операции `map` и `filter` применяются параллельно.

---

🧱 2. DataFrame

💡 Определение

**DataFrame** — это **высокоуровневая абстракция** над RDD,
представляющая **таблицу с именованными колонками**, как в SQL или Pandas.

DataFrame — это:

* *распределённый RDD + схема (schema)*,
* *удобный API*,
* *возможность использовать SQL-запросы и оптимизации*.

---

⚙️ Преимущества DataFrame перед RDD:

| Преимущество           | Описание                                                                          |
| ---------------------- | --------------------------------------------------------------------------------- |
| **Оптимизация**        | Использует *Catalyst Optimizer* — автоматически оптимизирует план вычислений      |
| **SQL-интеграция**     | Можно писать SQL-запросы (`df.select`, `df.filter`, `df.groupBy` или `spark.sql`) |
| **Удобство**           | Понятный интерфейс как у Pandas                                                   |
| **Производительность** | Более эффективное хранение и сериализация (через *Tungsten Engine*)               |
| **Совместимость**      | Работает с MLlib, Spark SQL, Hive, Parquet и др.                                  |

---

🔧 Пример DataFrame

```python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()

data = [("Alice", 25), ("Bob", 30), ("Cathy", 27)]
df = spark.createDataFrame(data, ["name", "age"])

df.show()
```

📊 Результат:

```
+-----+---+
| name|age|
+-----+---+
|Alice| 25|
|  Bob| 30|
|Cathy| 27|
+-----+---+
```

---

🚀 3. SparkSession

💡 Определение

**SparkSession** — это **главная точка входа** в PySpark.
Через неё создаются **DataFrame**, выполняются **SQL-запросы**,
и обеспечивается **доступ ко всем компонентам Spark** (Spark SQL, MLlib, Streaming и т.д.).

Можно сказать:

> `SparkSession` объединяет функциональность `SparkContext`, `SQLContext` и `HiveContext` в одном объекте.

---

🔧 Пример создания SparkSession

```python
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .getOrCreate()

# Создание DataFrame
df = spark.read.csv("data.csv", header=True, inferSchema=True)
df.show()

# SQL-запрос
df.createOrReplaceTempView("people")
spark.sql("SELECT name, age FROM people WHERE age > 25").show()
```

---

⚖️ Сравнение RDD, DataFrame и SparkSession

| Компонент        | Уровень | Что делает                                                              | Используется для                                               |
| ---------------- | ------- | ----------------------------------------------------------------------- | -------------------------------------------------------------- |
| **RDD**          | Низкий  | Хранит и обрабатывает распределённые коллекции объектов                 | Гибкая обработка данных, когда нужно контролировать вычисления |
| **DataFrame**    | Средний | Табличное представление данных с именованными колонками                 | SQL-анализ, агрегации, ETL, машинное обучение                  |
| **SparkSession** | Высокий | Точка входа в Spark-приложение, создаёт DataFrame, управляет контекстом | Управление Spark, подключение к источникам данных              |

---

🧠 Итог

* **RDD** — базовый строительный блок Spark: низкоуровневый, гибкий, но без автоматических оптимизаций.
* **DataFrame** — удобный и оптимизированный способ работы с данными, похожий на SQL и Pandas.
* **SparkSession** — единая точка входа, через которую создаются и управляются все Spark-объекты.

### Как устроена базовая архитектура Spark-приложения (Driver и Executors)?

⚙️ Базовая архитектура Spark-приложения: Driver и Executors

Apache Spark — это **распределённая система обработки данных**, построенная по принципу *master–worker*.  
В Spark-приложении есть два ключевых типа процессов:

- **Driver** — управляющий процесс (мозг приложения);  
- **Executors** — рабочие процессы, выполняющие вычисления.

---

🧠 1. Driver (Драйвер)

**Driver** — это центральный компонент, который управляет выполнением Spark-приложения.

Основные функции:
- Запускает **SparkSession** (или `SparkContext`);
- Читает код пользователя и строит **план вычислений (DAG — Directed Acyclic Graph)**;
- Разбивает задачу на этапы (*stages*) и подзадачи (*tasks*);
- Распределяет задачи между исполнителями (**Executors**);
- Следит за выполнением задач и собирает результаты;
- Предоставляет интерфейс мониторинга — **Spark UI**.

> 💡 Можно считать Driver "менеджером проекта", который планирует, кому и что нужно сделать.

---

⚙️ 2. Executors (Исполнители)

**Executors** — это рабочие процессы, выполняющие конкретные задачи, назначенные Driver-ом.

Основные функции:
- Выполняют задачи (*tasks*) на выделенных разделах данных (*partitions*);
- Хранят промежуточные данные в памяти (например, при кэшировании RDD);
- Возвращают результаты обратно Driver-у;
- Завершаются, когда приложение Spark заканчивает работу.

Каждый Executor работает **на отдельной ноде** кластера и может выполнять несколько задач параллельно (многопоточность).

> 💡 Executors — это как "работники", выполняющие конкретные вычислительные задачи.

---

🧩 3. Cluster Manager (Менеджер кластера)

Это компонент, который управляет ресурсами (CPU, RAM, ноды) и запускает Driver и Executors.

Поддерживаемые менеджеры:
| Менеджер | Описание |
|-----------|-----------|
| **Standalone** | Встроенный менеджер Spark |
| **YARN** | Менеджер кластера из Hadoop |
| **Mesos** | Универсальный кластерный менеджер |
| **Kubernetes** | Современный способ запуска Spark в контейнерах |

> Cluster Manager решает, где и сколько ресурсов выделить каждому компоненту.

---

🔄 Как работает Spark-приложение (пошагово)

1. **Пользователь** запускает приложение (например, `spark-submit`).
2. **Cluster Manager** выделяет ресурсы и запускает **Driver**.
3. **Driver**:
   - создает `SparkContext`;
   - строит план вычислений (DAG);
   - разбивает его на этапы и задачи;
   - отправляет их на **Executors**.
4. **Executors** выполняют задачи, обрабатывают данные и возвращают результаты.
5. **Driver** собирает результаты и отображает их пользователю или записывает в хранилище.

---

🧱 Архитектурная схема

```

+----------------------+

| Driver                   |
| ------------------------ |
| - Планирует DAG          |
| - Распределяет задачи    |
| - Собирает результаты    |
| +----------+-----------+ |

```
       |
       v
```

+----------------------+

| Cluster Manager          |
| ------------------------ |
| - Управляет ресурсами    |
| - Запускает процессы     |
| +----------+-----------+ |

```
       |
       v
```

+----------------------+     +----------------------+
|     Executor 1       | ... |     Executor N       |
|----------------------|     |----------------------|
| - Выполняет задачи   |     | - Выполняет задачи   |
| - Хранит данные RDD  |     | - Хранит данные RDD  |
+----------------------+     +----------------------+

```

---

🏁 Итог

| Компонент | Роль | Где работает | Основные функции |
|------------|------|---------------|------------------|
| **Driver** | Управляет вычислениями | На мастер-ноде | Планирует задачи, координирует работу |
| **Executor** | Выполняет задачи | На воркер-нодах | Обрабатывает данные, возвращает результат |
| **Cluster Manager** | Управляет ресурсами | На уровне кластера | Запускает и распределяет ресурсы |

### Что такое Apache Parquet и зачем он нужен?

🔍 Определение

**Apache Parquet** — это **колоночный формат хранения данных**, оптимизированный для **аналитических вычислений** и **больших данных**.  
Он был разработан в экосистеме Hadoop, но сейчас активно используется в Spark, Pandas, Hive, Presto, Dask, и многих других системах.

---

🧩 Основная идея

В обычных файлах (например, CSV) данные хранятся **построчно** — каждая строка содержит все значения сразу.  
В **Parquet** данные хранятся **по столбцам** (columnar storage).

📊 Пример:

CSV (построчно):
```

name, age, city
Alice, 25, London
Bob, 30, Paris
Charlie, 35, Berlin

```

Parquet (поколоночно):
```

name:    [Alice, Bob, Charlie]
age:     [25, 30, 35]
city:    [London, Paris, Berlin]

````

---

⚙️ Почему Parquet эффективен

| Преимущество | Описание |
|---------------|-----------|
| **1. Колоночное хранение** | При аналитических запросах часто нужны только некоторые столбцы, и Parquet позволяет читать только нужные колонки, экономя I/O. |
| **2. Сжатие и кодирование** | Данные в одном столбце обычно однотипные → хорошо сжимаются. Parquet автоматически использует эффективные алгоритмы сжатия (Snappy, Gzip, Brotli и др.). |
| **3. Поддержка типов данных** | Parquet сохраняет **типизацию** (int, string, timestamp и т.д.), что делает его безопаснее и быстрее, чем CSV или JSON. |
| **4. Совместимость с распределёнными системами** | Формат спроектирован для работы в Hadoop и Spark, с поддержкой параллельного чтения. |
| **5. Метаданные и статистика** | В каждом файле Parquet есть метаданные (min/max/кол-во строк), которые позволяют системам пропускать ненужные блоки данных при фильтрации (predicate pushdown). |

---

🚀 Зачем он нужен

**Apache Parquet** используется, когда нужно:
- хранить большие объёмы данных в компактном и удобном виде;
- выполнять **аналитические запросы** быстро и эффективно;
- интегрироваться с системами вроде **Apache Spark**, **Hive**, **Presto**, **Athena** или **BigQuery**;
- экономить место на диске и трафик при чтении.

---

💡 Пример использования в PySpark

```python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ParquetExample").getOrCreate()

# Чтение данных
df = spark.read.parquet("data/users.parquet")

# Обработка данных
filtered = df.filter(df.age > 30)

# Запись обратно в Parquet
filtered.write.parquet("data/adults.parquet")
````

🧠 Spark при этом читает только нужные колонки и строки, что делает обработку **в разы быстрее**, чем при использовании CSV.

---

🏁 Итог

| Характеристика                  | Parquet         | CSV            |
| ------------------------------- | --------------- | -------------- |
| Формат                          | Бинарный        | Текстовый      |
| Тип хранения                    | Колоночный      | Построчный     |
| Сжатие                          | Да (встроенное) | Обычно вручную |
| Типизация                       | Да              | Нет            |
| Скорость аналитических запросов | 🚀 Высокая      | 🐢 Низкая      |
| Использование в Big Data        | Да              | Ограничено     |

# Задание 0: Подготовка

Установите PySpark (pip install spark) и сделайте импорты

In [8]:
# ! pip install pyspark

In [259]:
import pyspark 
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col, avg, count, split, explode, lit, when, coalesce
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, LongType

Создайте Spark сессию

In [4]:
spark = ( SparkSession.builder 
.appName("MovieLens Lab") 
.master("local[*]") 
.getOrCreate() 
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/02 16:03:13 WARN Utils: Your hostname, users-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 172.20.10.4 instead (on interface en0)
25/11/02 16:03:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/02 16:03:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Отключите излишнее логирование

In [6]:
spark.sparkContext.setLogLevel("ERROR")

# Задание 1: Предобработка

Загрузите файлы movies.dat и ratings.dat в Spark DataFrame (Не пугайтесь непонятного формата файла, так все и трэба, вам просто нужно явно указать разделитель (::) и задать схему данных (названия колонок), так как в файлах нет заголовков. Используйте StructType и StructField. Это лучшая практика по сравнению с inferSchema)

In [36]:
base = "/Users/user/"

# ratings.dat
ratings_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("timestamp", LongType(), True)
])

ratings = (spark.read
          .option("delimiter", "::")
          .schema(ratings_schema)
          .csv(base + "ratings.dat"))

ratings.printSchema()
ratings.show(5, truncate=False)

# movies.dat
movies_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("genres", StringType(), True)
])

movies = (spark.read
          .option("delimiter", "::")
          .schema(movies_schema)
          .csv(base + "movies.dat"))

movies.printSchema()
movies.show(5, truncate=False)

# users.dat
users_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("occupation", IntegerType(), True)
])

users = (spark.read
          .option("delimiter", "::")
          .schema(users_schema)
          .csv(base + "users.dat"))

users.printSchema()
users.show(5, truncate=False)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: long (nullable = true)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|1     |1193   |5     |978300760|
|1     |661    |3     |978302109|
|1     |914    |3     |978301968|
|1     |3408   |4     |978300275|
|1     |2355   |5     |978824291|
+------+-------+------+---------+
only showing top 5 rows
root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+----------------------------------+----------------------------+
|movieId|title                             |genres                      |
+-------+----------------------------------+----------------------------+
|1      |Toy Story (1995)                  |Animation|Children's|Comedy |
|2      |Jumanji (1995)                    |Adventure|Children's|Fantasy|
|3      |Grumpie

После загрузки и перед началом любых трансформаций, закэшируйте DataFrame: ratings_df.cache(). Объясните, зачем это нужно

In [40]:
movies.cache()
users.cache()
ratings.cache()

DataFrame[userId: int, movieId: int, rating: int, timestamp: bigint]

**Зачем это нужно**:

Spark запоминает (кэширует) результаты вычисления этого DataFrame в памяти (RAM). Это означает: после первой операции (например, count(), show() или join), Spark сохранит вычисленные данные в оперативке — и при повторных обращениях не будет заново пересчитывать весь пайплайн.

Столбец “Жанры” в movies имеет неудобный формат: "Adventure|Animation|Children". Намного удобнее работать, если каждый жанр будет в отдельной строке. Разделите строку на массив по разделителю и “взорвите” массив.

In [70]:
movies.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Animation|Childre...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|        Comedy|Drama|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows


In [90]:
movies_split_df = movies.withColumn("genres_array", split(col("genres"), "\|"))

In [92]:
movies_split_df.show()

+-------+--------------------+--------------------+--------------------+
|movieId|               title|              genres|        genres_array|
+-------+--------------------+--------------------+--------------------+
|      1|    Toy Story (1995)|Animation|Childre...|[Animation, Child...|
|      2|      Jumanji (1995)|Adventure|Childre...|[Adventure, Child...|
|      3|Grumpier Old Men ...|      Comedy|Romance|   [Comedy, Romance]|
|      4|Waiting to Exhale...|        Comedy|Drama|     [Comedy, Drama]|
|      5|Father of the Bri...|              Comedy|            [Comedy]|
|      6|         Heat (1995)|Action|Crime|Thri...|[Action, Crime, T...|
|      7|      Sabrina (1995)|      Comedy|Romance|   [Comedy, Romance]|
|      8| Tom and Huck (1995)|Adventure|Children's|[Adventure, Child...|
|      9| Sudden Death (1995)|              Action|            [Action]|
|     10|    GoldenEye (1995)|Action|Adventure|...|[Action, Adventur...|
|     11|American Presiden...|Comedy|Drama|Romance|

In [94]:
movies_exploded_df = movies_split_df.withColumn("genre", explode(col("genres_array")))

In [98]:
movies_exploded_df = movies_exploded_df.select("movieId", "title", "genre")

In [100]:
movies_exploded_df.show(10, truncate=False)

+-------+------------------------+----------+
|movieId|title                   |genre     |
+-------+------------------------+----------+
|1      |Toy Story (1995)        |Animation |
|1      |Toy Story (1995)        |Children's|
|1      |Toy Story (1995)        |Comedy    |
|2      |Jumanji (1995)          |Adventure |
|2      |Jumanji (1995)          |Children's|
|2      |Jumanji (1995)          |Fantasy   |
|3      |Grumpier Old Men (1995) |Comedy    |
|3      |Grumpier Old Men (1995) |Romance   |
|4      |Waiting to Exhale (1995)|Comedy    |
|4      |Waiting to Exhale (1995)|Drama     |
+-------+------------------------+----------+
only showing top 10 rows


# Задание 2: Аналитика

Найдите 10 самых популярных жанров

In [107]:
top_10_genres = (movies_exploded_df
                .groupBy("genre")
                .agg(count("*").alias("count"))
                .orderBy(col("count").desc())
                .limit(10))

In [111]:
top_10_genres.show()

+----------+-----+
|     genre|count|
+----------+-----+
|     Drama| 1603|
|    Comedy| 1200|
|    Action|  503|
|  Thriller|  492|
|   Romance|  471|
|    Horror|  343|
| Adventure|  283|
|    Sci-Fi|  276|
|Children's|  251|
|     Crime|  211|
+----------+-----+



Найдите еще топ 10 чего-то на свой вкус

In [113]:
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
|     1|   3408|     4|978300275|
|     1|   2355|     5|978824291|
|     1|   1197|     3|978302268|
|     1|   1287|     5|978302039|
|     1|   2804|     5|978300719|
|     1|    594|     4|978302268|
|     1|    919|     4|978301368|
|     1|    595|     5|978824268|
|     1|    938|     4|978301752|
|     1|   2398|     4|978302281|
|     1|   2918|     4|978302124|
|     1|   1035|     5|978301753|
|     1|   2791|     4|978302188|
|     1|   2687|     3|978824268|
|     1|   2018|     4|978301777|
|     1|   3105|     5|978301713|
|     1|   2797|     4|978302039|
+------+-------+------+---------+
only showing top 20 rows


In [133]:
top_10_ratings = (ratings
    .groupBy("movieId")
    .agg(count(when(col("rating") == 5, True)).alias("count_5"))
    .orderBy(col("count_5").desc())
    .limit(10))

top_10_ratings.show()

+-------+-------+
|movieId|count_5|
+-------+-------+
|   2858|   1963|
|    260|   1826|
|   1198|   1500|
|   1196|   1483|
|    527|   1475|
|    858|   1475|
|    318|   1457|
|   2571|   1430|
|   2028|   1405|
|   2762|   1385|
+-------+-------+



In [151]:
(top_10_ratings
.join(movies, on="movieId", how="left")
.select("title", "count_5")).show()

+--------------------+-------+
|               title|count_5|
+--------------------+-------+
|American Beauty (...|   1963|
|Star Wars: Episod...|   1826|
|Raiders of the Lo...|   1500|
|Star Wars: Episod...|   1483|
|Schindler's List ...|   1475|
|Godfather, The (1...|   1475|
|Shawshank Redempt...|   1457|
|  Matrix, The (1999)|   1430|
|Saving Private Ry...|   1405|
|Sixth Sense, The ...|   1385|
+--------------------+-------+



# Задание 3: Объединение данных (Join)

Найти 10 самых популярных фильмов по количеству оценок

In [169]:
top_10_popular = (ratings
    .groupBy("movieId")
    .agg(count("rating").alias("count"))
    .orderBy(col("count").desc())
    .limit(10)
    .join(movies, on="movieId", how="left")
    .select("title", "count"))

top_10_popular.show(truncate=False)

+-----------------------------------------------------+-----+
|title                                                |count|
+-----------------------------------------------------+-----+
|American Beauty (1999)                               |3428 |
|Star Wars: Episode IV - A New Hope (1977)            |2991 |
|Star Wars: Episode V - The Empire Strikes Back (1980)|2990 |
|Star Wars: Episode VI - Return of the Jedi (1983)    |2883 |
|Jurassic Park (1993)                                 |2672 |
|Saving Private Ryan (1998)                           |2653 |
|Terminator 2: Judgment Day (1991)                    |2649 |
|Matrix, The (1999)                                   |2590 |
|Back to the Future (1985)                            |2583 |
|Silence of the Lambs, The (1991)                     |2578 |
+-----------------------------------------------------+-----+



Найдите 10 лучших фильмов (самый высокий средний рейтинг), у которых не менее 500 оценок

In [189]:
top_10_films_avg_rating = (ratings
    .groupBy("movieId")
    .agg(
        avg("rating").alias("avg_rating"),
        count("rating").alias("count_rating")
    )
    .filter(col("count_rating") >= 500)
    .join(movies, on="movieId", how="left")
    .select("title", "avg_rating", "count_rating")
    .orderBy(col("avg_rating").desc())
    .limit(10))

In [191]:
top_10_films_avg_rating.show(truncate=False)

+-------------------------------------------------------------------+-----------------+------------+
|title                                                              |avg_rating       |count_rating|
+-------------------------------------------------------------------+-----------------+------------+
|Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)|4.560509554140127|628         |
|Shawshank Redemption, The (1994)                                   |4.554557700942973|2227        |
|Godfather, The (1972)                                              |4.524966261808367|2223        |
|Close Shave, A (1995)                                              |4.52054794520548 |657         |
|Usual Suspects, The (1995)                                         |4.517106001121705|1783        |
|Schindler's List (1993)                                            |4.510416666666667|2304        |
|Wrong Trousers, The (1993)                                         |4.507936507936508|882 

Придумайте как вы можете осмысленно показать разницу между видами джоинов (inner, left, right, outer и cross). Используйте для этого маленькие тестовые таблицы

In [219]:
dataA = [(1, "Alice"), (2, "Bob"), (3, "Cathy")]
colsA = ["id", "name"]
A = spark.createDataFrame(dataA, colsA)

print("=== TABLE A ===")
A.show()

dataB = [(2, "Sales"), (3, "HR"), (4, "IT")]
colsB = ["id", "dept"]
B = spark.createDataFrame(dataB, colsB)

print("=== TABLE B ===")
B.show()

a = A.alias("a")
b = B.alias("b")

print("=== INNER JOIN ===")
inner = a.join(b, col("a.id") == col("b.id"), how="inner") \
         .select(col("a.id").alias("id"), col("a.name"), col("b.dept"))
inner.show()

print("=== LEFT JOIN ===")
left = a.join(b, col("a.id") == col("b.id"), how="left") \
        .select(col("a.id").alias("id"), col("a.name"), col("b.dept"))
left.show()

print("=== RIGHT JOIN ===")
right = a.join(b, col("a.id") == col("b.id"), how="right") \
         .select(coalesce(col("a.id"), col("b.id")).alias("id"), col("a.name"), col("b.dept"))
right.show()

print("=== FULL OUTER JOIN ===")
full = a.join(b, col("a.id") == col("b.id"), how="outer") \
        .select(coalesce(col("a.id"), col("b.id")).alias("id"),
                col("a.name"), col("b.dept")) \
        .orderBy("id")
full.show()

print("=== CROSS JOIN ===")
cross = a.crossJoin(b).select(col("a.id").alias("id_a"), col("a.name"), col("b.id").alias("id_b"), col("b.dept"))
cross.show()


=== TABLE A ===
+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|Cathy|
+---+-----+

=== TABLE B ===
+---+-----+
| id| dept|
+---+-----+
|  2|Sales|
|  3|   HR|
|  4|   IT|
+---+-----+

=== INNER JOIN ===
+---+-----+-----+
| id| name| dept|
+---+-----+-----+
|  2|  Bob|Sales|
|  3|Cathy|   HR|
+---+-----+-----+

=== LEFT JOIN ===
+---+-----+-----+
| id| name| dept|
+---+-----+-----+
|  1|Alice| NULL|
|  2|  Bob|Sales|
|  3|Cathy|   HR|
+---+-----+-----+

=== RIGHT JOIN ===
+---+-----+-----+
| id| name| dept|
+---+-----+-----+
|  2|  Bob|Sales|
|  3|Cathy|   HR|
|  4| NULL|   IT|
+---+-----+-----+

=== FULL OUTER JOIN ===
+---+-----+-----+
| id| name| dept|
+---+-----+-----+
|  1|Alice| NULL|
|  2|  Bob|Sales|
|  3|Cathy|   HR|
|  4| NULL|   IT|
+---+-----+-----+

=== CROSS JOIN ===


+----+-----+----+-----+
|id_a| name|id_b| dept|
+----+-----+----+-----+
|   1|Alice|   2|Sales|
|   1|Alice|   3|   HR|
|   1|Alice|   4|   IT|
|   2|  Bob|   2|Sales|
|   2|  Bob|   3|   HR|
|   2|  Bob|   4|   IT|
|   3|Cathy|   2|Sales|
|   3|Cathy|   3|   HR|
|   3|Cathy|   4|   IT|
+----+-----+----+-----+



# Задание 4: Spark SQL

Создайте временные таблицы из DataFrame

In [245]:
ratings.createOrReplaceTempView("ratings")
movies.createOrReplaceTempView("movies")

In [247]:
ratings

DataFrame[userId: int, movieId: int, rating: int, timestamp: bigint]

Выполните задание (топ-10 фильмов с >= 500 оценок), используя spark.sql()

In [250]:
top_10_movies_sql = spark.sql("""
    SELECT
        m.movieId,
        m.title,
        ROUND(AVG(r.rating), 2) AS avg_rating,
        COUNT(r.rating) AS num_ratings
    FROM ratings r
    JOIN movies m
        ON r.movieId = m.movieId
    GROUP BY m.movieId, m.title
    HAVING COUNT(r.rating) >= 500
    ORDER BY avg_rating DESC, num_ratings DESC
    LIMIT 10
""")

In [252]:
top_10_movies_sql.show()

+-------+--------------------+----------+-----------+
|movieId|               title|avg_rating|num_ratings|
+-------+--------------------+----------+-----------+
|   2019|Seven Samurai (Th...|      4.56|        628|
|    318|Shawshank Redempt...|      4.55|       2227|
|    858|Godfather, The (1...|      4.52|       2223|
|     50|Usual Suspects, T...|      4.52|       1783|
|    745|Close Shave, A (1...|      4.52|        657|
|    527|Schindler's List ...|      4.51|       2304|
|   1148|Wrong Trousers, T...|      4.51|        882|
|   1198|Raiders of the Lo...|      4.48|       2514|
|    904|  Rear Window (1954)|      4.48|       1050|
|    260|Star Wars: Episod...|      4.45|       2991|
+-------+--------------------+----------+-----------+



# Задание 5: Spark MLlib

PySpark имеет встроенную библиотеку машинного обучения (MLlib). Мы используем самый популярный алгоритм для рекомендаций — ALS (Alternating Least Squares, Метод попеременных наименьших квадратов).

Создайте модель ALS

In [267]:
als = ALS(
    maxIter=5,
    regParam=0.01,
    userCol="userId",     
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop"
)


Обучите модель на обучающей выборке (80%), сделайте предсказание на тестовой выборке (20%)

In [269]:
train, test = ratings.randomSplit([0.8, 0.2], seed=42)

In [271]:
model = als.fit(train)

In [273]:
predictions = model.transform(test)

Оцените качество по среднеквадратичной ошибке

In [275]:
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

In [277]:
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error (RMSE) на тестовой выборке = {rmse}")

[Stage 457:>                                                        (0 + 1) / 1]

Root-mean-square error (RMSE) на тестовой выборке = 0.8948603710989897


# Задание 6: Персонализация рекомендаций

Выберите пользователя (id) и получите топ 10 фильмов для него.
Используйте метод .recommendForUserSubset(... , 10)

In [298]:
user_id = 1
user_df = spark.createDataFrame([(user_id,)], ["userId"])

user_recommendations = model.recommendForUserSubset(user_df, 10)

exploded = user_recommendations.select(
    col("userId"),
    explode(col("recommendations")).alias("rec")
)

exploded = exploded.select(
    col("userId"),
    col("rec.movieId").alias("movieId"),
    col("rec.rating").alias("predicted_rating")
)

top_movies = exploded.join(movies, on="movieId").select(
    "userId", "movieId", "title", "predicted_rating"
).orderBy(col("predicted_rating").desc())

top_movies.show(truncate=False)

+------+-------+---------------------------------------+----------------+
|userId|movieId|title                                  |predicted_rating|
+------+-------+---------------------------------------+----------------+
|1     |2765   |Acid House, The (1998)                 |8.850419        |
|1     |2426   |Theory of Flight, The (1998)           |8.442208        |
|1     |1181   |Shooter, The (1995)                    |7.5269437       |
|1     |2984   |On Any Sunday (1971)                   |7.395924        |
|1     |1585   |Love Serenade (1996)                   |7.3863907       |
|1     |718    |Visitors, The (Les Visiteurs) (1993)   |6.893715        |
|1     |3900   |Crime and Punishment in Suburbia (2000)|6.8858147       |
|1     |3240   |Big Tease, The (1999)                  |6.780712        |
|1     |443    |Endless Summer 2, The (1994)           |6.745242        |
|1     |1000   |Curdled (1996)                         |6.702002        |
+------+-------+----------------------

# Завершите работу спарк сессии и расскажите зачем это делать

In [300]:
spark.stop()

Почему это важно:

Освобождение ресурсов
Spark создаёт множество потоков и соединений с кластером или локальными ресурсами (CPU, память, драйвер, executors). Если не завершить сессию, они будут продолжать висеть в памяти и могут мешать другим процессам.

Избежание конфликтов
В одной сессии могут остаться временные таблицы, кэшированные данные и параметры конфигурации. Если потом создать новую сессию без остановки старой, это может вызвать неожиданные ошибки.

Чистота логов и файлов
Spark может писать временные файлы и логи в каталог пользователя. При корректном завершении сессии эти файлы удаляются или закрываются корректно.

Хорошая практика
Особенно если работа идёт в Jupyter Notebook: при многократном запуске ячеек с Spark без spark.stop() каждая новая ячейка создаёт новую сессию, что может перегрузить систему.